In [1]:
#!pip install mysql.connector

In [2]:
#!pip install Flask
#!pip install flask_ngrok

In [3]:
#!pip install flask flask-sqlalchemy flask-login

In [4]:
#!pip install bs4


In [5]:
#import lib
from flask import Flask
from flask import Flask, render_template,redirect,url_for
from flask import Blueprint
from flask import request
from flask import flash
from flask import session
import secrets
import flask
import requests


In [6]:
import mysql.connector 
from mysql.connector import errorcode
import mysql.connector
#connection db config
config = {
  'user': 'web_207109277',
  'password': '207109277',
  'host': '34.96.189.105',
  'database': 'cookbook',
  'raise_on_warnings': True
}
try:
  mydb = mysql.connector.connect(**config)
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)

In [ ]:
app = Flask(__name__)
app.secret_key = secrets.token_urlsafe(16)
@app.route("/")
def home():
  return render_template('login.html')

@app.route("/userlogin",methods=["POST"])
def userlogin():
    name = request.form.get('username')
    password = request.form.get('password')
    sql =  "SELECT * FROM user WHERE name = %s AND password = %s"    
    mycursor = mydb.cursor()
    mycursor.execute(sql,(name,password,))
    myresult = mycursor.fetchall()
    if not myresult: 
      flash('This is account not exits')
      return redirect(url_for('home'))
    mydb.commit()    
    session['userID'] = myresult[0][0]#user id 
    return  redirect(url_for('index'))

@app.route("/register",methods=["POST","GET"])
def register():
    if request.method =='GET':
        return render_template('register.html')
    else:
        name = request.form.get('username')
        password = request.form.get('password')    
        sql = "SELECT * FROM user WHERE name = %s"
        mycursor = mydb.cursor()      
        mycursor.execute(sql,(name,))
        myresult = mycursor.fetchall()
        if myresult:
            flash('This is account alredy exits')
            return redirect(url_for('index'))
        else:
          sql = "INSERT INTO user(name,password) VALUES ( %s, %s)"
          mycursor.execute(sql,(name,password,))
          mydb.commit()
          #get new user id 
          sql =  "SELECT * FROM user WHERE name = %s AND password = %s"
          mycursor = mydb.cursor()
          mycursor.execute(sql,(name,password,))
          myresult = mycursor.fetchall()
          session['userID'] = myresult[0][0]#user id 
          return redirect(url_for('index'))




@app.route("/user_logout",methods=["POST","GET"])
def logout():
         session["userID"] = None
         return render_template('login.html')    
        
@app.route("/profile",methods=["POST","GET"])
def user_profile():
    if request.method =='GET':
        userid=flask.escape(session['userID'])
        sql="SELECT cookbook.cookbookID,user.name,cookbook.title,cookbook.description,cookbook.detail FROM cookbook,user where cookbook.user_id = user.userid and cookbook.user_id = %s;"
        mycursor = mydb.cursor()      
        mycursor.execute(sql,(str(userid),))
        myresult = mycursor.fetchall()
        mydb.commit()
        return render_template('profile.html',data=myresult)
    else:         
        password = request.form.get('password')
        userid=flask.escape(session['userID'])
        new_password= request.form.get('new_password')
        sql = "SELECT * FROM user WHERE userid = %s"
        mycursor = mydb.cursor()      
        mycursor.execute(sql,(str(userid),))
        myresult = mycursor.fetchall()
        mydb.commit()
        user_old_password=myresult[0][2]#old password   
        if user_old_password != password:
            flash('The password is not match!')
            return redirect(url_for('user_profile'))
        else:
            sql="UPDATE user SET password= %s where userid= %s"
            mycursor=mydb.cursor()
            mycursor.execute(sql,(str(new_password),str(userid),))
            mydb.commit()
            flash('The password is changed!')
            return redirect(url_for('logout'))

@app.route("/index")
def index():
        sql="SELECT cookbook.cookbookID,user.name,cookbook.title,cookbook.description,cookbook.detail FROM cookbook,user where cookbook.user_id = user.userid;"
        mycursor=mydb.cursor()
        mycursor.execute(sql)
        myresult=mycursor.fetchall()
        mydb.commit()    
        return render_template('index.html',data=myresult)
    





@app.route("/add_Myf",methods=["POST","GET"])
def add_Myf():
        f_id=request.args.get('id')
        userid=flask.escape(session['userID']) 
        sql = "INSERT INTO favorite(userid,cookbookID) VALUES (%s,%s)"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(userid),str(f_id),))
        mydb.commit()
        flash('Successfully Added')
        return redirect(url_for('index'))
 
@app.route("/myFavorite",methods=["POST","GET"])
def index_Myf():
        userid=flask.escape(session['userID'])
        sql = " SELECT favorite.favoriteID,cookbook.cookbookID,title,user.name,description FROM favorite,cookbook,user where favorite.userid= %s and cookbook.cookbookID=favorite.cookbookID and user.userid=cookbook.user_id"         
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(userid),))
        myresult=mycursor.fetchall()
        mydb.commit() 
        return render_template('myFavorite.html',data=myresult)


@app.route("/del_Myf",methods=["POST","GET"])
def del_Myf():
        f_id=request.args.get('id')
        userid=flask.escape(session['userID'])
        sql = " DELETE FROM favorite  WHERE  userid = %s and favoriteID= %s"         
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(userid),str(f_id),))
        mydb.commit()
        flash('Successfully Deleted')
        return redirect(url_for('index'))

@app.route("/detail",methods=["POST","GET"])
def detail():
        b_id=request.args.get('id')
        sql="SELECT cookbookID,user.name,title,description,detail FROM cookbook,user where cookbook.user_id =user.userid and cookbook.cookbookID = %s;"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(b_id),))
        myresult=mycursor.fetchall()
        mydb.commit()       
        return render_template('detail.html',data=myresult)


 
@app.route("/DeleteCookbook",methods=["POST","GET"])
def del_Cookbook():
        b_id=request.args.get('id')
        userid=flask.escape(session['userID'])
        sql = " DELETE FROM cookbook  WHERE  cookbookID = %s and user_id= %s"         
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(b_id),str(userid),))
        mydb.commit()
        flash('Successfully Deleted')
        return redirect(url_for('user_profile'))   

@app.route("/uploadCookbook",methods=["POST","GET"])
def uploadCookbook():
        if request.method =='GET':
            return render_template('uploadCookbook.html')
        else: 
            title=request.form.get('title')
            destription=request.form.get('destription')
            detail=request.form.get('detail')
            userid=flask.escape(session['userID'])
            sql = "INSERT INTO cookbook(user_id,title,description,detail) VALUES ( %s, %s,%s, %s)"            
            mycursor=mydb.cursor()
            mycursor.execute(sql,(str(userid),str(title),str(destription),str(detail),))
            mydb.commit()
            flash('Successfully Created')
            return redirect(url_for('user_profile'))   

@app.route("/editCookbook",methods=["POST","GET"])
def editCookbook():
        if request.method =='GET':
            b_id=request.args.get('id')
            return render_template('editCookbook.html',data=b_id)
        else: 
            title=request.form.get('title')
            destription=request.form.get('destription')
            detail=request.form.get('detail')
            cookbookID=request.form.get('cookbookID')
            sql="UPDATE cookbook SET title= %s,description=%s,detail=%s where cookbookID= %s"            
            mycursor=mydb.cursor()
            mycursor.execute(sql,(str(title),str(destription),str(detail),str(cookbookID),))
            mydb.commit()
            flash('Successfully Updated!')
            return redirect(url_for('user_profile')) 




app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Jun/2021 00:19:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2021 00:19:27] "GET /static/css/animate.min.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:19:27] "GET /static/css/swiper.min.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:19:27] "GET /static/css/normalize.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:19:27] "GET /static/css/owl.carousel.min.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:19:27] "GET /static/css/meanmenu.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:19:27] "GET /static/css/slick.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:19:27] "GET /static/css/jquery.fancybox.min.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:19:27] "GET /static/css/jquery-ui.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:19:27] "GET /static/css/nice-select.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:19:31] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2021 00:19:31] "G

127.0.0.1 - - [05/Jun/2021 00:20:20] "GET /static/css/jquery.fancybox.min.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:20:20] "GET /static/css/jquery-ui.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:20:20] "GET /static/css/nice-select.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:20:38] "POST /uploadCookbook HTTP/1.1" 302 -
127.0.0.1 - - [05/Jun/2021 00:20:38] "GET /profile HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2021 00:20:38] "GET /static/css/owl.carousel.min.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:20:38] "GET /static/css/slick.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:20:38] "GET /static/css/normalize.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:20:38] "GET /static/css/meanmenu.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:20:38] "GET /static/css/swiper.min.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:20:38] "GET /static/css/animate.min.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:20:38] "GET /static/css/jquery.fancybox.min.css HTTP/1.

127.0.0.1 - - [05/Jun/2021 00:21:12] "GET /static/css/swiper.min.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:21:12] "GET /static/css/slick.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:21:12] "GET /static/css/owl.carousel.min.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:21:12] "GET /static/css/jquery.fancybox.min.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:21:12] "GET /static/css/jquery-ui.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:21:12] "GET /static/css/nice-select.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:21:12] "GET /profile HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2021 00:21:13] "GET /myFavorite HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2021 00:21:13] "GET /static/css/normalize.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:21:13] "GET /static/css/owl.carousel.min.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:21:13] "GET /static/css/slick.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:21:13] "GET /static/css/meanmenu.css HTTP/1.1" 404 -
127.0

127.0.0.1 - - [05/Jun/2021 00:22:07] "GET /static/css/animate.min.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:22:07] "GET /static/css/owl.carousel.min.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:22:07] "GET /static/css/normalize.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:22:07] "GET /static/css/meanmenu.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:22:07] "GET /static/css/swiper.min.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:22:07] "GET /static/css/slick.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:22:07] "GET /static/css/jquery.fancybox.min.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:22:07] "GET /static/css/jquery-ui.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:22:07] "GET /static/css/nice-select.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:22:07] "GET /index HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2021 00:22:12] "GET /static/css/normalize.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2021 00:22:12] "GET /static/css/animate.min.css HTTP/1.1